In [18]:
from torf import Torrent
import hashlib

In [19]:
tor = Torrent.read("pic.torrent")

In [20]:
# Extract the name of the file
file_name = tor.name
print("File Name:", file_name)

# Extract the size of the file
file_size = tor.size
print("File Size:", file_size)

# Extract the number of files in the torrent
num_files = len(tor.files)
print("Number of Files:", num_files)

# Extract the trackers used in the torrent
trackers = tor.trackers
print("Trackers:", trackers)

File Name: turquoise_mountains_4k.jpg
File Size: 5367782
Number of Files: 1
Trackers: []


In [21]:
import binascii

tor.hashes

# Convert the SHA1 piece hashes to human-readable format
hashes_readable = [binascii.hexlify(hash).decode() for hash in tor.hashes]

# Print the human-readable hashes
print("Hashes (Human-readable):", hashes_readable)

Hashes (Human-readable): ['c0b677c042356e25b3b7204fe9bb2734b192d37f', 'b01261a663a5a9373e1b59cd36a91736beff0587', '584af26332a386d0dca40b5081838b7ba4019fd8', '2df2f275b6a9acad460ef1af5d357aaa20271f71', '7d6ad1b46a651ec2a102f6c7321ac685005d857d', 'e3314e85fceb07ef41070e90c422e1ce29e808d6', 'e90ba6e2a2805efc7e555f7fc1bc2d714c699f7f', '2f105140259b6707fea615cfc479a66f5ba52969', 'd8047b5db4befd5e3fdef6416a68f414dfbaf3f6', 'b1bd4222df3e7711b278635942b47f9181cc091f', '4ddd7538bc1ada97145fe2f418581e498846f09b']


In [22]:
tor.pieces

11